<a href="https://colab.research.google.com/github/diazdela/Semantic-Segmentation-of-Breast-Cancer-Tissue/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project - Breast Cancer Semantic Segmentation

## Dataset https://github.com/CancerDataScience/CrowdsourcingDataset-Amgadetal2019.git

In [ ]:
# Clone the repository
!git clone https://github.com/CancerDataScience/CrowdsourcingDataset-Amgadetal2019.git
%cd CrowdsourcingDataset-Amgadetal2019

# Install the required packages
!pip install girder_client pillow numpy scikit-image imageio

!python download_crowdsource_dataset.py


In [ ]:
# Identifying Directories
import os
os.getcwd()

# Parent Directory
parent_dir = "/content/CrowdsourcingDataset-Amgadetal2019"

# Sub-directories
sub_dirs = [d for d in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, d))]
print(sub_dirs)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Visualize Images
img_sub_dir = "images"

# Path to the sub-directory
img_sub_path = os.path.join(parent_dir, img_sub_dir)

# List files in sub-directory
img_sub_files = os.listdir(img_sub_path)

# Print the list of files in the sub-directory
print(img_sub_files)

# List files in the sub-directory
img_file_1 = img_sub_files[1]
file_path = os.path.join(img_sub_path, img_file_1)

# Visualize
image = Image.open(file_path)
height=480
img_1 = image.resize((int(float(image.size[0])*height/float(image.size[1])),height))

plt.rcParams["figure.figsize"] = (5,5)
plt.imshow(img_1); plt.show()

## Model https://github.com/IanTaehoonYoo/semantic-segmentation-pytorch

In [ ]:
!pip install seg-torch

In [ ]:
from torchvision import transforms

from segmentation.data_loader.segmentation_dataset import SegmentationDataset
from segmentation.data_loader.transform import Rescale, ToTensor
from segmentation.trainer import Trainer
from segmentation.predict import *
from segmentation.models import all_models
from util.logger import Logger

train_images = r'/content/CrowdsourcingDataset-Amgadetal2019/images'
train_labled = r'/content/CrowdsourcingDataset-Amgadetal2019/masks'

if __name__ == '__main__':
    model_name = "fcn8_vgg16"
    device = 'cuda'
    batch_size = 4
    n_classes = 34
    num_epochs = 10
    image_axis_minimum_size = 200
    pretrained = True
    fixed_feature = False
    image_target_size = (200, 200)

    logger = Logger(model_name=model_name, data_name='example')

    ### Loader
    compose = transforms.Compose([
        transforms.Resize(image_target_size),  # Resize images to a consistent size
        ToTensor()
         ])

    train_datasets = SegmentationDataset(train_images, train_labled, n_classes, compose)
    train_loader = torch.utils.data.DataLoader(train_datasets, batch_size=batch_size, shuffle=True, drop_last=True)

#    test_datasets = SegmentationDataset(test_images, test_labeled, n_classes, compose)
#    test_loader = torch.utils.data.DataLoader(test_datasets, batch_size=batch_size, shuffle=True, drop_last=True)

    ### Model
    model = all_models.model_from_name[model_name](n_classes, batch_size,
                                                   pretrained=pretrained,
                                                   fixed_feature=fixed_feature)
    model.to(device)

    ###Load model
    ###please check the foloder: (.segmentation/test/runs/models)
    #logger.load_model(model, 'epoch_15')

    ### Optimizers
    if pretrained and fixed_feature: #fine tunning
        params_to_update = model.parameters()
        print("Params to learn:")
        params_to_update = []
        for name, param in model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t", name)
        optimizer = torch.optim.Adadelta(params_to_update)
    else:
        optimizer = torch.optim.Adadelta(model.parameters())


    ### Train
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    trainer = Trainer(model, optimizer, logger, num_epochs, train_loader)
    trainer.train()

